# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
import os


In [7]:
os.getcwd

<function nt.getcwd>

In [8]:
df_raw = pd.read_csv("C:\\Users\\stuar\\dat6melb_students\\lesson-11\\assets\\dataset\\collegeadmissions.csv")
df = df_raw.dropna() 
df.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [9]:
# frequency table for prestige and whether or not someone was admitted
my_tab = pd.crosstab(index=df['admit'],  # Make a crosstab
                              columns="count")      # Name the count column

my_tab

col_0,count
admit,
0,273
1,127


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [10]:
# create dummy variables

rank_dummy = pd.get_dummies(df['rank'], prefix='rank')
rank_dummy.head(10)

,rank_1,rank_2,rank_3,rank_4
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1
5,0,1,0,0
6,1,0,0,0
7,0,1,0,0
8,0,0,1,0
9,0,1,0,0


In [11]:
# fuse back
df_clean = pd.concat([df, rank_dummy], axis = 1)

# drop original rank column
df_clean = df_clean.drop(columns = ['rank'])
df_clean.head(10)

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1
5,1,760,3.00,0,1,0,0
6,1,560,2.98,1,0,0,0
7,0,400,3.08,0,1,0,0
8,1,540,3.39,0,0,1,0
9,0,700,3.92,0,1,0,0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: I'm not really sure I understand this question. If we're referring to dummying rank as above, then we need 1 column for each possible variable (e.g. Rank 1, 2, 3, 4). 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [21]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(df['admit'], df['rank'], normalize='columns', margins=True)

rank,1,2,3,4,All
admit,,,,,
0,0.459016,0.642384,0.768595,0.820896,0.6825
1,0.540984,0.357616,0.231405,0.179104,0.3175


The crosstab (which I've changed to show as % of column total rather than raw values) shows that the higher the prestige rank of an applicant's undergrad college, the greater their chance of admission

#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [23]:
.54/(1-.54)

1.173913043478261

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [75]:
# Probability = sum of admission where rank = 2,3 or 4 / the total applications 
df[(df['admit']==1) & (df['rank']!=1)].count() / df[df['rank']!=1].count()

admit    0.277286
gre      0.277286
gpa      0.277286
rank     0.277286
dtype: float64

The <b>probability</b> of admission for someone in a rank 2-4 college is 27.7%.

In [78]:
# Odds = sum of admission where rank = 2,3 or 4 over the sum of non-admissions where rank =2,3 or 4 
df[(df['admit']==1) & (df['rank']!=1)].count() / df[(df['admit']==0) & (df['rank']!=1)].count()

admit    0.383673
gre      0.383673
gpa      0.383673
rank     0.383673
dtype: float64

The <b>odds</b> of admission for someone in a rank 2-4 college is 27.7%.

#### 3.3 Calculate the odds ratio

In [79]:
# Divide the odds of admission for someone who attended a rank 1 college over 
# the odds of admission for someone that did not attend a rank 1 college
df[(df['admit']==1) & (df['rank']==1)].count() / df[(df['admit']==0) & (df['rank']==1)].count()

admit    1.178571
gre      1.178571
gpa      1.178571
rank     1.178571
dtype: float64

In [80]:
df[(df['admit']==1) & (df['rank']!=1)].count() / df[(df['admit']==0) & (df['rank']!=1)].count()

admit    0.383673
gre      0.383673
gpa      0.383673
rank     0.383673
dtype: float64

In [81]:
1.178571 / 0.383673

3.071811151683856

#### 3.4 Write this finding in a sentenance: 

Answer: The odds ratio of admission for an applicant who attended a rank 1 college is 3.07

#### 3.5 Print the cross tab for prestige_4

In [87]:
#create a crosstab showing the probability of attendance for rank 4 college applicants
pd.crosstab(df['admit'], df['rank']==4, normalize='index', margins=True)

rank,False,True
admit,,
0,0.798535,0.201465
1,0.905512,0.094488
All,0.832500,0.167500


#### 3.6 Calculate the OR 

In [88]:
.094488 / .905512

0.10434759561441484

#### 3.7 Write this finding in a sentence

Answer: The odds ratio of admission for an applicant who attended a rank 4 college is .104

## Part 4. Analysis

In [148]:
# i've used statsmodels instead of scikit party to try something different and also because scikit 
# doesn't seem to have a summary output like the R equivalent
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt

In [132]:
# workaround for the missing chisqprob in scipy.stats
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

In [133]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
clean_df = df[cols_to_keep].join(rank_dummy.loc[:, 'rank_1':])
clean_df.head(2)

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

#### 4.1 Set the covariates to a variable called train_cols

In [144]:
clean_df = clean_df.drop(columns = 'intercept')

In [145]:
X = clean_df.drop(columns = ['admit'])
y = clean_df['admit']
X.head(2)

,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,380,3.61,0,0,1,0
1,660,3.67,0,0,1,0


#### 4.2 Fit the model

In [146]:
# fit the classifier# fit t 
res = sm.Logit(y, X).fit()

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6


#### 4.3 Print the summary results

In [147]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  400
Model:                          Logit   Df Residuals:                      394
Method:                           MLE   Df Model:                            5
Date:                Mon, 07 May 2018   Pseudo R-squ.:                 0.08292
Time:                        22:35:19   Log-Likelihood:                -229.26
converged:                       True   LL-Null:                       -249.99
                                        LLR p-value:                 7.578e-08
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gre            0.0023      0.001      2.070      0.038       0.000       0.004
gpa            0.8040      0.332      2.423      0.015       0.154       1.454
rank_1        -3.9900      1.140     -3.500      0.000      -6.224      -1.756
rank_2        -4.6654      1.109     -4.205      0.000      -6.840      -2.491
rank_3        -5.3302      1.150     -4.637      0.000      -7.583      -3.077
rank_4        -5.5414      1.138     -4.869      0.000      -7.772      -3.311
==============================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [137]:
params = res.params
conf = res.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print(np.exp(conf))

            2.5%     97.5%        OR
gre     1.000120  1.004418  1.002267
gpa     1.166122  4.281877  2.234545
rank_1  0.001981  0.172783  0.018500
rank_2  0.001070  0.082819  0.009415
rank_3  0.000509  0.046091  0.004843
rank_4  0.000421  0.036485  0.003921


#### 4.5 Interpret the OR of Prestige_2

Answer: Applicants who attended a rank_2 college had lower odds of admission than those attending rank_1 but higher than those attending rank_3 and rank_4.

#### 4.6 Interpret the OR of GPA

Answer: Each point incrase in GPA increased the odds of admission by 2.23

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [149]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [157]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(df_clean['gre'].min(), df_clean['gre'].max(), 10, dtype='float')
gres

array([220.        , 284.44444444, 348.88888889, 413.33333333,
       477.77777778, 542.22222222, 606.66666667, 671.11111111,
       735.55555556, 800.        ])

In [163]:
gpas = np.linspace(df_clean['gpa'].min(), df_clean['gpa'].max(), 10, dtype='float')
gpas

array([2.26      , 2.45333333, 2.64666667, 2.84      , 3.03333333,
       3.22666667, 3.42      , 3.61333333, 3.80666667, 4.        ])

In [167]:
# have tried trouble shooting this for a while but can't get it to work - have tried changing 
#various values to integers and so on but no luck
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

TypeError: slice indices must be integers or None or have an __index__ method

#### 5.1 Recreate the dummy variables

In [ ]:
# recreate the dummy variables

# keep only what we need for making predictions


#### 5.2 Make predictions on the enumerated dataset

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.